# Test Run 11 Change to Categorical
This notebook continues from test run 10. Specifically, we <b>change the original 5 binary classification scheme to categorical</b>. We follow the tutorial ([Categorial_Classification](../../tutorials/advanced/Categorical_Classes.ipynb)) for inspiration. 


In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))
debug_mode=1 # if not in debug mode, please set debug_mode=0, find more info about debug mode in test run 9 maybe?
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

### Section 1: Loading documents and train/dev/test segments. 

#### The following two celsl should load in many papers if they haven't been loaded. 

In [2]:
import os
os.environ['SNORKELDB'] = "postgresql:///abstractnet"
print(os.environ['SNORKELDB'])

from snorkel import SnorkelSession
from snorkel.parser import TSVDocPreprocessor
session = SnorkelSession()

# # Here, we just set how many documents we'll process for automatic testing- you can safely ignore this!
n_docs = 90000 # this is the upper limit of number of docs
doc_preprocessor = TSVDocPreprocessor('data/slim-s2-corpus-00_70kpaper_061418_cleaned_noBookLecture_10cscw_2k_order_preserving_top45000.tsv', encoding="utf-8",max_docs=n_docs)

from snorkel.parser.spacy_parser import Spacy
from snorkel.parser import CorpusParser

# corpus_parser = CorpusParser(parser=Spacy())
# %time corpus_parser.apply(doc_preprocessor, count=n_docs)# ,parallelism=5)

from snorkel.models import Document, Sentence  # defined in context.py file

print("Documents:", session.query(Document).count())

postgresql:///abstractnet
snorkel_conn_string postgresql:///abstractnet
if snorkel_postgres? True


/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


Documents: 44999


In [4]:
from snorkel import SnorkelSession
from snorkel.parser.spacy_parser import Spacy
from snorkel.parser import CorpusParser
from snorkel.models import Document, Sentence

session = SnorkelSession()
print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())

docs = session.query(Document).all()
sents = session.query(Sentence).all()  # get all sentences from snorkel.db
n_max_corpus=0
for sent in sents:
    n_max_corpus=max(n_max_corpus,len(sent.words))
print("The longest sentence has "+str(n_max_corpus)+" tokens.")

train_sents = set()
dev_sents   = set()
test_sents  = set()

train_doc_set=set()
dev_doc_set = set()
test_doc_set = set()

for i, doc in enumerate(docs):
    for s in doc.sentences:
#         if doc.name[:7]=="2K_dev_":
        if "semanticDBLP_" in doc.name and doc.name[-1]=="3":
            dev_sents.add(s)
            dev_doc_set.add(doc.name)
        elif doc.name[:8]=="2K_test_":
            test_sents.add(s)
            test_doc_set.add(doc.name)
#         else:
#         elif "semanticDBLP_" in doc.name:
#             train_sents.add(s)
#             train_doc_set.add(doc.name)
print("train_doc_set length",len(train_doc_set))
print("dev_doc_set length",len(dev_doc_set))
print("dev_sents length", len(dev_sents),"dev_doc_set size", len(dev_doc_set), "avg n_sent per doc","%.3f"%(float(len(dev_sents))/float(len(dev_doc_set))))
print("test_doc_set length",len(test_doc_set))
print("test_sents length", len(test_sents),"test_doc_set size", len(test_doc_set), "avg n_sent per doc","%.3f"%(float(len(test_sents))/float(len(test_doc_set))))

Documents: 44999
Sentences: 319134
The longest sentence has 700 tokens.
train_doc_set length 0
dev_doc_set length 2774
dev_sents length 19365 dev_doc_set size 2774 avg n_sent per doc 6.981
test_doc_set length 0


ZeroDivisionError: float division by zero

In [6]:
from snorkel.models import candidate_subclass
from snorkel.candidates import Ngrams, CandidateExtractor
from snorkel.matchers import *
import datetime

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
    
### NOTE: this extract_and_display is different from earlier test run notebook's version, whrere selected_split=0 will only extract train_segments. 
def extract_and_display(matcher,candidate_class,candidate_class_name,train_breakdown_map=None,dev_doc_breakdown_map=None,selected_split=0,is_print=True):  # split over train/dev/test but returns only train set
    for (i, sents) in ([(0,train_sents)] if selected_split==0 else ([(2,test_sents)] if selected_split==2 else [(1,dev_sents)])):
        %time matcher.apply(sents, split=i)
        printmd("**Split "+str(i)+" - number of candidates extracted: "+str(session.query(candidate_class).filter(candidate_class.split == i).count())+"**\n\n")
    
    train_cands = session.query(candidate_class).filter(candidate_class.split == selected_split).all()
    dev_cands = session.query(candidate_class).filter(candidate_class.split == 1).all()
    test_cands=session.query(candidate_class).filter(candidate_class.split==2).all()
    
    return train_cands,dev_cands,test_cands

Segment = candidate_subclass('Segment', ['segment_cue'],values=['PP','MN','NULL'])
ngrams = Ngrams(n_max=n_max_corpus) 
non_tilde_matcher=DictionaryMatch(d=['~'],longest_match_only=True,reverse=True)  
non_tilde_segment_extractor=CandidateExtractor(Segment, [ngrams], [non_tilde_matcher])

non_comma_matcher=DictionaryMatch(d=[','],longest_match_only=True,reverse=True)  
non_comma_segment_extractor=CandidateExtractor(Segment, [ngrams], [non_comma_matcher])

train_doc_breakdown_map=dict()
dev_doc_breakdown_map=dict()

# print(datetime.datetime.now())
# train_segments,dev_segments,test_segments=extract_and_display(non_comma_segment_extractor,Segment,"Segment",train_breakdown_map=train_doc_breakdown_map,dev_doc_breakdown_map=dev_doc_breakdown_map,is_print=False,selected_split=0)
# print(datetime.datetime.now())
train_segments,dev_segments,test_segments=extract_and_display(non_comma_segment_extractor,Segment,"Segment",train_breakdown_map=train_doc_breakdown_map,dev_doc_breakdown_map=dev_doc_breakdown_map,is_print=False,selected_split=1)
# train_segments,dev_segments,test_segments=extract_and_display(non_tilde_segment_extractor,Segment,"Segment",train_breakdown_map=train_doc_breakdown_map,dev_doc_breakdown_map=dev_doc_breakdown_map,is_print=False,selected_split=2)

train_segments=session.query(Segment).filter(Segment.split == 0).all()
print("len(train_segments)",len(train_segments))
dev_segments=session.query(Segment).filter(Segment.split == 1).all()
print("len(dev_segments)",len(dev_segments))
test_segments=session.query(Segment).filter(Segment.split == 2).all()
print("len(test_segments)",len(test_segments))

Clearing existing...
snorkel_conn_string postgresql:///abstractnet
if snorkel_postgres? True
Running UDF...
snorkel_conn_string postgresql:///abstractnet
if snorkel_postgres? True
[========================================] 100%

IntegrityError: (psycopg2.IntegrityError) duplicate key value violates unique constraint "segment_segment_cue_id_key"
DETAIL:  Key (segment_cue_id)=(435478) already exists.
 [SQL: 'INSERT INTO segment (id, segment_cue_id, segment_cue_cid) VALUES (%(id)s, %(segment_cue_id)s, %(segment_cue_cid)s)'] [parameters: ({'id': 621592, 'segment_cue_id': 435478, 'segment_cue_cid': None}, {'id': 621593, 'segment_cue_id': 812202, 'segment_cue_cid': None}, {'id': 621594, 'segment_cue_id': 568402, 'segment_cue_cid': None}, {'id': 621595, 'segment_cue_id': 440805, 'segment_cue_cid': None}, {'id': 621596, 'segment_cue_id': 681317, 'segment_cue_cid': None}, {'id': 621597, 'segment_cue_id': 631981, 'segment_cue_cid': None}, {'id': 621598, 'segment_cue_id': 870189, 'segment_cue_cid': None}, {'id': 621599, 'segment_cue_id': 870188, 'segment_cue_cid': None}  ... displaying 10 of 37380 total bound parameter sets ...  {'id': 658970, 'segment_cue_id': 943357, 'segment_cue_cid': None}, {'id': 658971, 'segment_cu

**Split 1 - number of candidates extracted: 0**



len(train_segments) 621591
len(dev_segments) 0
len(test_segments) 0


In [7]:
print(type(train_segments))

<class 'list'>


In [27]:
print(Candidate.__dict__)
cands=session.query(Candidate).filter(Candidate.id%100<5).all()
session.query(Candidate).filter(Candidate.split==1).delete()
print(len(cands))
print(cands[0].__dict__)

{'__module__': 'snorkel.models.candidate', '__doc__': '\n    An abstract candidate relation.\n\n    New relation types should be defined by calling candidate_subclass(),\n    **not** subclassing this class directly.\n    ', '__tablename__': 'candidate', 'id': <sqlalchemy.orm.attributes.InstrumentedAttribute object at 0x10b9261a8>, 'type': <sqlalchemy.orm.attributes.InstrumentedAttribute object at 0x10b926360>, 'split': <sqlalchemy.orm.attributes.InstrumentedAttribute object at 0x10b926410>, '__mapper_args__': {'polymorphic_identity': 'candidate', 'polymorphic_on': Column('type', String(), table=<candidate>, nullable=False)}, 'get_contexts': <function Candidate.get_contexts at 0x10b9256a8>, 'get_parent': <function Candidate.get_parent at 0x10b925730>, 'get_cids': <function Candidate.get_cids at 0x10b9257b8>, '__len__': <function Candidate.__len__ at 0x10b925840>, '__getitem__': <function Candidate.__getitem__ at 0x10b9258c8>, '__repr__': <function Candidate.__repr__ at 0x10b925950>, '__

### Section 2: Create generative labels for Purpose on train set

Any LFs for other aspects will be grouped into `neg_for_purpose_LFs`

In [28]:
from snorkel.lf_helpers import *
from snorkel.annotations import LabelAnnotator
from LF.util_purpose_default_categorical import common_LFs,purpose_LFs   # neg_LFs are reverse LFs for Background, Mechanism, Method, and Finding
from LF.util_mechanism_default_categorical import mechanism_LFs

from LF.util_background_default_categorical import background_LFs
from LF.util_method_default_categorical import method_LFs
from LF.util_finding_default_categorical import finding_LFs

print("common_LFs",len(common_LFs))
print("purpose_LFs",len(purpose_LFs))
print("mechanism_LFs",len(mechanism_LFs))
# print([lf.__name__ for lf in mechanism_LFs])
print("method_LFs",len(method_LFs))
print("background_LFs",len(background_LFs))
print("finding_LFs",len(finding_LFs))


common_LFs 0
purpose_LFs 32
mechanism_LFs 34
method_LFs 3
background_LFs 13
finding_LFs 19


### Section 3: Repeat Section 2 for all other 4 aspects! 

In [37]:
from snorkel.learning import GenerativeModel
from util import get_candidate_text
import matplotlib.pyplot as plt
import datetime

# maybe write a cids_query session.query(Candidate.id).filter(Candidate.split == split) from def apply(self, split=0, key_group=0, replace_key_set=True, cids_query=None,
#         **kwargs):
def get_L_train(LFs,parallelism=1,split=0):
    L_train=None
    labeler=None
    np.random.seed(1701)
    labeler = LabelAnnotator(lfs=LFs)
    print(datetime.datetime.now())
    L_train = labeler.apply(cids_query=session.query(Candidate.id).filter(Candidate.id%100>95),split=split,parallelism=parallelism)
    print(datetime.datetime.now())
    print(type(L_train))
    print(L_train.shape)
    printmd("**Total non_overlapping_coverage on L_train (percentage of labelled over all)**  "+str(L_train.non_overlapping_coverage()))
    return L_train

def get_train_marginals(L_train,step_size_nominator=0.1,epochs=100,threads=1,class_prior=False,decay=1.0):
    try:
        gen_model=None
        print(datetime.datetime.now())
        gen_model = GenerativeModel(lf_propensity=True,class_prior=class_prior)
#         gen_model.train(L_train, cardinality=3, epochs=100, decay=0.95, step_size=step_size_nominator/L_train.shape[0], reg_param=0.0,verbose=False,threads=threads) # try step_size = 1/ next time? 
        gen_model.train(L_train, cardinality=3, epochs=epochs, decay=decay,step_size=step_size_nominator/1e2,reg_param=1e-6,verbose=True,threads=threads)
        print("training finished ")
        print(datetime.datetime.now())
        print("Finished training generative model, now checking performance against development set labels...")
        print("Calculating marginals!")
        train_marginals = gen_model.marginals(L_train)  # the marginal probability of each candidate being True
#         assert np.all(train_marginals.sum(axis=1) - np.ones(train_marginals.shape[0]) < 1e-10)
        print("no exception")
        return train_marginals
    except:
        print("exception! ")
        return train_marginals


In [38]:
L_train=get_L_train(common_LFs+purpose_LFs+mechanism_LFs+background_LFs+method_LFs+finding_LFs)

snorkel_conn_string postgresql:///abstractnet
if snorkel_postgres? True
2018-10-03 22:13:28.051650
snorkel_conn_string postgresql:///abstractnet
if snorkel_postgres? True
cids_count 24860
key_group 0
Clearing existing...
snorkel_conn_string postgresql:///abstractnet
if snorkel_postgres? True
Running UDF...
snorkel_conn_string postgresql:///abstractnet
if snorkel_postgres? True
[========================================] 100%

2018-10-03 22:18:11.012213
<class 'snorkel.annotations.csr_LabelMatrix'>
(24860, 100)


**Total non_overlapping_coverage on L_train (percentage of labelled over all)**  0.8572003218020917

In [55]:
from snorkel.utils import sparse_nonzero
import scipy.sparse as sparse

L_train_segments=session.query(Candidate).filter(Candidate.id%100>95).all()
L_nonzero=sparse_nonzero(L_train)
not_all_zero_row_idx=sorted(set(sparse.find(L_nonzero)[0]))
# not_all_zero_row_idx=[int(i) for i in not_all_zero_row_idx]
print(type(L_train[not_all_zero_row_idx]))
print("not_all_zero_row_idx has ",len(not_all_zero_row_idx))
print(type(L_train_segments))

L_train_trim=L_train[not_all_zero_row_idx]
print(type(not_all_zero_row_idx[0]))
L_train_segments=[L_train_segments[i] for i in not_all_zero_row_idx]
print(L_train_trim.shape)
# print(L_train.todense())


<class 'snorkel.annotations.csr_AnnotationMatrix'>
not_all_zero_row_idx has  21310
<class 'list'>
<class 'numpy.int32'>
(21310, 100)


In [2]:
# help(L_train_trim) #.lf_stats(session)
import pandas as pd 
pd.set_option('display.max_rows', 500)
from IPython.display import display, HTML
lf_temp = display(L_train_trim.lf_stats(session))

NameError: name 'L_train_trim' is not defined

In [61]:
dense=L_train_trim.todense()

# print(type(L_train_trim[:,:5]))
# lf_names=[L_train_trim.get_key(session, j).name for j in range(L_train_trim.shape[1])]

# print("\n".join([str(idx)+" "+name for idx, name in enumerate(lf_names)]))
# print(lf_names)
# print(lf_names.index('LF_mechanism_PRP_VBP'))
# index_range=list(range(57,62))+list(range(92,L_train.shape[1]))
# print(lf_names[[57:62]+[92:100]])
# print(index_range)

# index_range_new=sorted(index_range+[10,12,16,19,28,29,31,32,39,56,63,64,65,66,68,72,73,74,75,90,91])
# import operator
# print(operator.itemgetter(*index_range)(lf_names))
# print(lf_names[index_range])
print([(dense==i).sum() for i in range(4)])

# print(type(L_train[:,index_range]))
# L_trim_column=L_train[:cutoff,index_range_new]
# dense=L_trim_column.todense()
# print([(dense==i).sum() for i in range(4)])

[2071060, 35073, 22765, 2102]


In [41]:
# L_train=get_L_train(common_LFs+purpose_LFs+mechanism_LFs+background_LFs+method_LFs+finding_LFs)
train_marginals=get_train_marginals(L_train_trim,threads=18,epochs=50)

print(train_marginals)
print(train_marginals.sum(axis=1))
assert np.all(train_marginals.sum(axis=1) - np.ones(train_marginals.shape[0]) < 1e-10)

winHighlight=train_marginals.argmax(axis=1)
print("PP",(winHighlight==0).sum())
print("MN",(winHighlight==1).sum())
print("NULL",(winHighlight==2).sum())


2018-10-03 22:22:57.229750
Compiles variable matrix
LF label variables -- initial loop to set all variables over 0
LF label variables -- initial loop to set all variables over 10000
LF label variables -- initial loop to set all variables over 20000
LF label variables -- initial loop to set all variables over 30000
LF label variables -- initial loop to set all variables over 40000
LF label variables -- initial loop to set all variables over 50000
L_coo.nnz 220504
LF labels -- now set the non-zero labels 0
LF labels -- now set the non-zero labels 10000
LF labels -- now set the non-zero labels 20000
LF labels -- now set the non-zero labels 30000
LF labels -- now set the non-zero labels 40000
LF labels -- now set the non-zero labels 50000
LF labels -- now set the non-zero labels 60000
LF labels -- now set the non-zero labels 70000
LF labels -- now set the non-zero labels 80000
LF labels -- now set the non-zero labels 90000
LF labels -- now set the non-zero labels 100000
LF labels -- now se

FACTOR 0: EPOCH #1
Current stepsize = 0.001
Learning epoch took 11.206 sec.
Weights:
    weightId: 0
        isFixed: True
        weight:  1.0

    weightId: 1
        isFixed: False
        weight:  -0.057916799537346304

    weightId: 2
        isFixed: True
        weight:  1.0

    weightId: 3
        isFixed: False
        weight:  -0.017920152066545385

    weightId: 4
        isFixed: True
        weight:  1.0

    weightId: 5
        isFixed: False
        weight:  1.1840422461338198

    weightId: 6
        isFixed: True
        weight:  1.0

    weightId: 7
        isFixed: False
        weight:  -0.15491613922785064

    weightId: 8
        isFixed: True
        weight:  1.0

    weightId: 9
        isFixed: False
        weight:  -0.14091792450394328

    weightId: 10
        isFixed: True
        weight:  1.0

    weightId: 11
        isFixed: False
        weight:  -0.1539170151891934

    weightId: 12
        isFixed: True
        weight:  1.0

    weightId: 13
        

FACTOR 0: EPOCH #2
Current stepsize = 0.001
Learning epoch took 11.473 sec.
Weights:
    weightId: 0
        isFixed: True
        weight:  1.0

    weightId: 1
        isFixed: False
        weight:  -0.4428815016303393

    weightId: 2
        isFixed: True
        weight:  1.0

    weightId: 3
        isFixed: False
        weight:  -0.45988552941745203

    weightId: 4
        isFixed: True
        weight:  1.0

    weightId: 5
        isFixed: False
        weight:  0.6169716536105037

    weightId: 6
        isFixed: True
        weight:  1.0

    weightId: 7
        isFixed: False
        weight:  -0.6438654856307141

    weightId: 8
        isFixed: True
        weight:  1.0

    weightId: 9
        isFixed: False
        weight:  -0.5818706516856963

    weightId: 10
        isFixed: True
        weight:  1.0

    weightId: 11
        isFixed: False
        weight:  -0.7458594352498011

    weightId: 12
        isFixed: True
        weight:  1.0

    weightId: 13
        isFix

FACTOR 0: EPOCH #3
Current stepsize = 0.001
Learning epoch took 11.219 sec.
Weights:
    weightId: 0
        isFixed: True
        weight:  1.0

    weightId: 1
        isFixed: False
        weight:  -0.39483186896954525

    weightId: 2
        isFixed: True
        weight:  1.0

    weightId: 3
        isFixed: False
        weight:  -0.3598369350930002

    weightId: 4
        isFixed: True
        weight:  1.0

    weightId: 5
        isFixed: False
        weight:  0.8668800953251223

    weightId: 6
        isFixed: True
        weight:  1.0

    weightId: 7
        isFixed: False
        weight:  -0.6537882454635954

    weightId: 8
        isFixed: True
        weight:  1.0

    weightId: 9
        isFixed: False
        weight:  -0.5228044598190807

    weightId: 10
        isFixed: True
        weight:  1.0

    weightId: 11
        isFixed: False
        weight:  -0.6377755152614706

    weightId: 12
        isFixed: True
        weight:  1.0

    weightId: 13
        isFix

FACTOR 0: EPOCH #4
Current stepsize = 0.001
Learning epoch took 11.389 sec.
Weights:
    weightId: 0
        isFixed: True
        weight:  1.0

    weightId: 1
        isFixed: False
        weight:  -0.27579078729086104

    weightId: 2
        isFixed: True
        weight:  1.0

    weightId: 3
        isFixed: False
        weight:  -0.2947971892808625

    weightId: 4
        isFixed: True
        weight:  1.0

    weightId: 5
        isFixed: False
        weight:  0.9677691259323291

    weightId: 6
        isFixed: True
        weight:  1.0

    weightId: 7
        isFixed: False
        weight:  -0.6247115480251382

    weightId: 8
        isFixed: True
        weight:  1.0

    weightId: 9
        isFixed: False
        weight:  -0.49074407088642824

    weightId: 10
        isFixed: True
        weight:  1.0

    weightId: 11
        isFixed: False
        weight:  -0.6326994807637719

    weightId: 12
        isFixed: True
        weight:  1.0

    weightId: 13
        isFi

...

FACTOR 0: EPOCH #46
Current stepsize = 0.001
Learning epoch took 12.657 sec.
Weights:
    weightId: 0
        isFixed: True
        weight:  1.0

    weightId: 1
        isFixed: False
        weight:  0.8266340877572435

    weightId: 2
        isFixed: True
        weight:  1.0

    weightId: 3
        isFixed: False
        weight:  0.7602185758090924

    weightId: 4
        isFixed: True
        weight:  1.0

    weightId: 5
        isFixed: False
        weight:  2.12787671022075

    weightId: 6
        isFixed: True
        weight:  1.0

    weightId: 7
        isFixed: False
        weight:  -0.17881274291842528

    weightId: 8
        isFixed: True
        weight:  1.0

    weightId: 9
        isFixed: False
        weight:  0.10062734124822426

    weightId: 10
        isFixed: True
        weight:  1.0

    weightId: 11
        isFixed: False
        weight:  -0.4658267383991961

    weightId: 12
        isFixed: True
        weight:  1.0

    weightId: 13
        isFixed:

FACTOR 0: EPOCH #47
Current stepsize = 0.001
Learning epoch took 13.300 sec.
Weights:
    weightId: 0
        isFixed: True
        weight:  1.0

    weightId: 1
        isFixed: False
        weight:  0.8785315340825639

    weightId: 2
        isFixed: True
        weight:  1.0

    weightId: 3
        isFixed: False
        weight:  0.8081243853263371

    weightId: 4
        isFixed: True
        weight:  1.0

    weightId: 5
        isFixed: False
        weight:  2.212616284704477

    weightId: 6
        isFixed: True
        weight:  1.0

    weightId: 7
        isFixed: False
        weight:  -0.1797911454976259

    weightId: 8
        isFixed: True
        weight:  1.0

    weightId: 9
        isFixed: False
        weight:  0.08861583988970435

    weightId: 10
        isFixed: True
        weight:  1.0

    weightId: 11
        isFixed: False
        weight:  -0.477770425370402

    weightId: 12
        isFixed: True
        weight:  1.0

    weightId: 13
        isFixed: 

    weightId: 235
        isFixed: False
        weight:  -8.043243728720375

    weightId: 236
        isFixed: False
        weight:  -6.467377642764398

    weightId: 237
        isFixed: False
        weight:  -6.292811839455008

    weightId: 238
        isFixed: False
        weight:  -7.092490826261439

    weightId: 239
        isFixed: False
        weight:  -7.22821460654781

    weightId: 240
        isFixed: False
        weight:  -6.2967189834605595

    weightId: 241
        isFixed: False
        weight:  -6.7654681057867325

    weightId: 242
        isFixed: False
        weight:  -7.206220428031634

    weightId: 243
        isFixed: False
        weight:  -8.279634780346855

    weightId: 244
        isFixed: False
        weight:  -6.021484583991692

    weightId: 245
        isFixed: False
        weight:  -7.868561181044827

    weightId: 246
        isFixed: False
        weight:  -7.183285245900271

    weightId: 247
        isFixed: False
        weight:  -7.60

FACTOR 0: EPOCH #49
Current stepsize = 0.001
Learning epoch took 13.009 sec.
Weights:
    weightId: 0
        isFixed: True
        weight:  1.0

    weightId: 1
        isFixed: False
        weight:  0.8793191907391102

    weightId: 2
        isFixed: True
        weight:  1.0

    weightId: 3
        isFixed: False
        weight:  0.778934920866293

    weightId: 4
        isFixed: True
        weight:  1.0

    weightId: 5
        isFixed: False
        weight:  2.234095222848149

    weightId: 6
        isFixed: True
        weight:  1.0

    weightId: 7
        isFixed: False
        weight:  -0.15175062442052334

    weightId: 8
        isFixed: True
        weight:  1.0

    weightId: 9
        isFixed: False
        weight:  0.14558708191593805

    weightId: 10
        isFixed: True
        weight:  1.0

    weightId: 11
        isFixed: False
        weight:  -0.477655006737845

    weightId: 12
        isFixed: True
        weight:  1.0

    weightId: 13
        isFixed: 

        weight:  -7.7929015993423185

    weightId: 249
        isFixed: False
        weight:  -8.861666874021397

    weightId: 250
        isFixed: False
        weight:  -8.424478759435994

    weightId: 251
        isFixed: False
        weight:  -7.792901147748142

    weightId: 252
        isFixed: False
        weight:  -8.033275383857552

    weightId: 253
        isFixed: False
        weight:  -8.058258253988633

    weightId: 254
        isFixed: False
        weight:  -8.378515730842079

    weightId: 255
        isFixed: False
        weight:  -6.263098891509191

    weightId: 256
        isFixed: False
        weight:  -7.2204237665168485

    weightId: 257
        isFixed: False
        weight:  -5.15632265066732

    weightId: 258
        isFixed: False
        weight:  -6.654993956802103

    weightId: 259
        isFixed: False
        weight:  -5.800522952261409

    weightId: 260
        isFixed: False
        weight:  -5.304863342846468

    weightId: 261
        

Let's check if the generative labels makes any sense. -> Seems okay. 

In [43]:
pair_list=list(zip(L_train_segments[not_all_zero_row_idx],list(train_marginals[:,0].reshape(-1))))
# print(pair_list[:5])
pair_list.sort(key = lambda t: t[1],reverse=True)

print(pair_list[:5])

TypeError: only integer scalar arrays can be converted to a scalar index

### Section 3: writing soft generative noisy labels

In [31]:
from collections import defaultdict
from util import get_candidate_text
import pandas as pd

# initialize some variables
# df = pd.DataFrame([], columns=['winningHighlight', 'paperID', 'globalPsn','content'])    
aggregated_doc=defaultdict(lambda:[])
aggregated_sent=defaultdict(lambda:[])
aspects_list=["Purpose","Mechanism","Others"]

for idx,train_segment in enumerate(train_segments):
    
    sent_id=train_segment.get_parent().id    
    winningHighlight=list(train_marginals[idx])
    aggregated_sent[sent_id]+=[(train_segment.segment_cue.char_start,train_segment.segment_cue.char_end,get_candidate_text(train_segment),winningHighlight)]
    
    doc_id=train_segment.get_parent().get_parent().name
    if doc_id not in aggregated_doc:
        aggregated_doc[doc_id]=train_segment.get_parent().get_parent().sentences
        

df_list=[]      
for doc_id in aggregated_doc:
    globalPsn=1
    for sent in aggregated_doc[doc_id]:
        sent_id=sent.id
        if sent_id not in aggregated_sent:
            raise ValueError('we have no info about this sent'+str(sent))
        aggregated_sent[sent_id]=sorted(aggregated_sent[sent_id], key=lambda x: x[0])
        for idx,pair in enumerate(aggregated_sent[sent_id]):
            text=pair[2]
            for idx2,token in enumerate(text.split()):
                if len(token.strip())==0:
                    continue
                df_list+=[[pair[3],doc_id,globalPsn,token.strip(".")]]
                globalPsn+=1
            if idx!=len(aggregated_sent[sent_id])-1:
                df_list+=[[pair[3],doc_id,globalPsn,","]]
            else:
                df_list+=[[pair[3],doc_id,globalPsn,"."]]
            globalPsn+=1
                
                
df=pd.DataFrame(df_list,columns=['winningHighlight', 'paperID', 'globalPsn','content'])

df.to_csv(open("data/annotations_train_5560_word-level_categorical.csv","w"))

In [14]:
docid=1
print("clause\t",train_segments[docid],"\n")
print("sent id\t",train_segments[docid].get_parent().id,"\n")
print("sentence\t",train_segments[docid].get_parent(),"\n")
print("sentence_dict\t",train_segments[docid].get_parent().__dict__,"\n")
print("all sentences in the doc\t",train_segments[docid].get_parent().get_parent().sentences,"\n")
print("this sentence pos in doc\t",train_segments[docid].get_parent().get_parent().sentences.index(train_segments[docid].get_parent()),"\n")
# print(docs[1].sentences)
print("doc name\t", train_segments[docid].get_parent().get_parent().name)

clause	 Segment(Span("b'cello'", sentence=19479, chars=[96,100], words=[15,15])) 

sent id	 19479 

sentence	 Sentence(Document 58cca85cc50f90cdd388252e,0,b'Abstract.\xe2\x80\x9cThe Watercourse Way\xe2\x80\x9d is a mostly-music interactive multimedia performance for violin, cello, percussion, and dancer.') 

sentence_dict	 {'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1165ea048>, 'stable_id': '58cca85cc50f90cdd388252e::sentence:0:126', 'dep_parents': [3, 3, 5, 3, 0, 12, 9, 9, 12, 12, 12, 5, 12, 13, 14, 14, 16, 16, 18, 18, 18, 5], 'pos_tags': ['DT', 'NNP', 'NNP', '-RRB-', 'VBZ', 'DT', 'RB', 'HYPH', 'NN', 'JJ', 'NN', 'NN', 'IN', 'NN', ',', 'NN', ',', 'NN', ',', 'CC', 'NN', '.'], 'abs_char_offsets': [0, 14, 26, 29, 31, 34, 36, 42, 43, 49, 61, 72, 84, 88, 94, 96, 101, 103, 113, 115, 119, 125], 'entity_cids': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'char_offsets': [0, 14, 26, 29, 31, 34, 36, 42,